![title](../data/imgs/boosting13.png)

Como vimos, Gradient Boosted Decision Trees é uma tecnica de Boosting que utiliza Stumped Decision Trees para a melhora gradual do modelo.

Mas como que isso é feito?

![title](../data/imgs/boosting12.png)

Esse ciclo é repetido de formas sucessivas enquanto construímos novos modelos e fazemos a devida combinação com nosso modelo de `Ensemble`. Nós iniciamos o ciclo ao calculuar os erros para cada valor do nosso dataset de treino. Então, nós construímos um novo modelo para fazer a previsão em cima desses erros e adicionamos essa previsão-de-erro ao nosso "ensemble de modelo".

Para fazer a previsão, nós adicionamos as predições de todos os modelos anteriores. Essas previsões são usadas para calcular novos modelos e então, as acoplamos no nosso ensemble.

Confuso? Para entender um pouco melhor a ideia, vamos relembrar a ideia de Regressão Linear

## Relembrando Regressão Linear

Na aula de Regressão Linear, vocês viram um método analítico (dos mínimos quadrados). Ele funciona bem com datasets pequeno, mas temos alguns problemas como datasets maiores. 

Na regressão, quando queremos definir a curva que define nosso modelo, nós podemos fazê-lo por meio do seguinte algoritmo:

- Ter um modelo $F_1(x)$ que faça a predição dos nossos dados
- Medir os nossos resíduos (erros): $h(x) = y - F(x)$
- Criar um novo modelo: $F_2(x) = F_1(x) + h_1(x)$

Podemos repetir isso $M$ vezes de forma a irmos combinando cada vez mais modelos:
![title](../data/imgs/boosting14.png)

Só que nós queremos minimizar os erros. Podemos fazer isso definindo uma função de custo que define o nosso erro, por exemplo: 

$f(x) = \sum_{i=1}^N{L(y_i,h(x_i))}$, ou seja, todos os nossos dados de treinamento. Aqui, então, $L$ pode ser o erro quadrático $L = (y_i - h(x_i))$

E para minimizar nós introduzimos a ideia de gradiente:

Para um dado ponto x do nosso dataset, queremos reduzí-lo ao mínimo local:

![title](../data/imgs/boosting15.png)


Ou seja, queremos fazer o nossa previsão ter seu erro cada vez mais reduzido, de forma que ele seja **muito próximo** de 0.

Matemátiquês a parte, o que está acontecendo é basicamente isso:

$x_i = x_{i-1} - \eta\frac{d_f(x_{i-1})}{dx}$, em que $\eta$ representa a _intensidade_ que queremos fazer esse ajuste em função do erro

![](../data/imgs/gradient_descent_example.gif)

A diferença do GD para Gradient Boost é que no GD vamos atualizando ponto a ponto, por interação. No caso do Gradient Boosting, nós não temos uma forma clara da forma da nossa função: ela é apenas um amontoado de árvore, em que cada árvore é atualizada pr interação. No caso do Gradient Descent, após a iteração _i_, você têm a atualização do ponto _i_ analizado. Para árvores, após a _i_ ésima interação, teremos _i_ árvores.

## O Dataset

Vamos dar uma explorada no nosso dataset

O que ele quer dizer para a gente?


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

data = pd.read_csv('../data/housing/inputs/train.csv')
data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = data.SalePrice
X = data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])
train_X, test_X, train_y, test_y = train_test_split(X.as_matrix(), y.as_matrix(), test_size=0.25)

my_imputer = Imputer()
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)

In [2]:
import xgboost as xgb

In [3]:
X.shape

(1460, 37)

In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Data columns (total 37 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1452 non-null float64
BsmtFinSF1       1460 non-null int64
BsmtFinSF2       1460 non-null int64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null int64
1stFlrSF         1460 non-null int64
2ndFlrSF         1460 non-null int64
LowQualFinSF     1460 non-null int64
GrLivArea        1460 non-null int64
BsmtFullBath     1460 non-null int64
BsmtHalfBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
F

In [5]:
X.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,1460.0,730.500000,421.610009,1.0,365.75,730.5,1095.25,1460.0
MSSubClass,1460.0,56.897260,42.300571,20.0,20.00,50.0,70.00,190.0
LotFrontage,1201.0,70.049958,24.284752,21.0,59.00,69.0,80.00,313.0
LotArea,1460.0,10516.828082,9981.264932,1300.0,7553.50,9478.5,11601.50,215245.0
OverallQual,1460.0,6.099315,1.382997,1.0,5.00,6.0,7.00,10.0
OverallCond,1460.0,5.575342,1.112799,1.0,5.00,5.0,6.00,9.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.00,1973.0,2000.00,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.00,1994.0,2004.00,2010.0
MasVnrArea,1452.0,103.685262,181.066207,0.0,0.00,0.0,166.00,1600.0
BsmtFinSF1,1460.0,443.639726,456.098091,0.0,0.00,383.5,712.25,5644.0


In [6]:
data_corr = data.corr()['SalePrice'].sort_values(ascending=False)

In [7]:
data_corr[(data_corr>0.5) & (data_corr!=1)]

OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
GarageArea      0.623431
TotalBsmtSF     0.613581
1stFlrSF        0.605852
FullBath        0.560664
TotRmsAbvGrd    0.533723
YearBuilt       0.522897
YearRemodAdd    0.507101
Name: SalePrice, dtype: float64

In [9]:
my_model = xgb.XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(train_X, train_y, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [10]:
# make predictions
predictions = my_model.predict(test_X)

from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))

Mean Absolute Error : 16273.7593964


In [11]:
my_model.score(test_X,test_y)

0.89471478108331226

## Seleção de Hiperparâmetros

O XGBoost têm alguns hiperparâmetros que podem afetar muito a acurácia do seu modelo e velocidade de treinamento.

Os dois primeiros são:
 - n_estimators 
 - early_stopping_rounds

#### Número de estimadores

Número de estimadores podem resultar em `underfitting` enquanto números elevados podem levar a `overfitting`. Novamente, caímos aqui no dilema entre complexidade e generalização de treinamento. Você deve experimentar com o dataset para achar os valores ideiais. Um número mágico (para ajudar) variam de 100 a 1000 mas depende muito do __learning rate__, que já vamos falar.

#### Early stopping rounds

Esse argumento, por sua vez, fornece uma forma de encontrar o valor ideal automaticamente. Ele faz com que o modelo pare de iteragir quando o score de validação seja concluido, mesmo que o número de estimadores não tenha atingido o número ideal. Aqui, é ideal usar um valor alto para o número de estimadores e, então, usar o early_stopping_rounds para encontrar um tempo ótimo para parar de iterar. Como por aleatoriedade, o val score pode não melhoreas, queremos um número de rounds seguidos em que não há melhora para considerarmos que nosso modelo deva parar.
Um valor bom aqui é val_score = 5


In [20]:
#Use 5 early stop roungs, n estimator as 1000
# early stop belongs to fit and n estimator belongs to 1000
my_model = xgb.XGBRegressor(n_estimators=1000)
my_model.fit(train_X, train_y, early_stopping_rounds=5, 
             eval_set=[(test_X, test_y)], verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [21]:
my_model.score(test_X, test_y)

0.89490544671428707

#### learning_rate

Aqui é exatamente o **$\eta$** que vimos lá em cima quando falamos do Gradient Descent. E, como ele determina a intensidade, ele é uma das peças chaves para atingir os melhores modelos do XGBoost

Na prática, ele serve para controlar o _overfitting_ do nosso modelo.

Assim, não precisamos nos preocupar muito com um número máximo de árvores. Se usarmos early_stopping, o número apropriado de árvores vai ser encontrado de forma automática, evitando o overfit.

Sendo assim, é melhor um learning rate alto ou baixo?

In [29]:
%time
#based on this, complete, including learning_rate
my_model = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.03)
my_model.fit(train_X, train_y, early_stopping_rounds=5, 
             eval_set=[(test_X, test_y)], verbose=False)
my_model.score(test_X, test_y)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.34 µs


0.88907980517731566

In [34]:
%time
#based on this, complete, including learning_rate
my_model = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.03, n_jobs=8)
my_model.fit(train_X, train_y, early_stopping_rounds=5, 
             eval_set=[(test_X, test_y)], verbose=False)
my_model.score(test_X, test_y)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


0.88907980517731566

#### n_jobs

Em datasets grandes, em que o tempo de execução é levado em consideração, você pode fazer uso de paralelism para que seus modelos sejam executados mais rapidamente. É comum setar o número de jobs (n_jobs) exatamente igual ao número de cores na sua máquina. Em datasets pequenos, contudo, isso não faz muita diferença

O XGBoost tem vários outros parâmetros que valem a pena serem discutidos, mas para um primeiro momento, esses são os ideais.

#### objective_functions

Objective Functions (ou Loss Functions) são as funções que vimos nos exemplos. Elas quantificam o quão longe nossa predição está do resultado real

Em outras palavras, queremos, minimizar essa função

No xgboost elas recebem os seguintes nomes:
 - reg:linear para problemas de regressão
 - reg:logistic usados em problemas de classificação quando você só quer a classe, não a chance de ser ela
 - reg: binary usados em problemas de classificacao quando você quer a chance de pertencer a cada classe
 